# Packages imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.neighbors import NearestNeighbors
import string 
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
from sklearn.preprocessing import MinMaxScaler

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth',None)

%load_ext autoreload
%autoreload 2

# Data Import

In [2]:
df_deals_raw = pd.read_excel('../raw_data/deals_raw.xlsx')
key_match = pd.read_excel('../raw_data/new_keywords.xlsx')
invest_key = pd.read_excel('../raw_data/invest_profile_keywords.xlsx')

In [3]:
matching_table = pd.read_excel('../raw_data/matching_table.xlsx')

# Preprocessing of DataFrames and training Models

## Functions

In [4]:
def remove_punctuations(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
    return text

## Targets

In [22]:
rel_columns = ['target_name', 'name', 'target_ebit', 'target_ebitda', 'target_revenue']

dict_2 = {}
dict_3 = {}
dict_4 = {}

for i in df_deals_raw['target_name'].unique():
    df_rel = df_deals_raw.loc[df_deals_raw['target_name'] == i]
    set_rel_2 = set(df_rel['name_de.2'])
    dict_2[i] = set_rel_2

    set_rel_3 = set(df_rel['name_de.3'])
    dict_3[i] = set_rel_3

    set_rel_4 = set(df_rel['name_de.4'])
    dict_4[i] = set_rel_4
    
df_deals_raw_rel = df_deals_raw[rel_columns].copy().drop_duplicates(subset = 'name')
df_deals_raw_rel['2'] = df_deals_raw_rel.target_name.apply(lambda x: dict_2.get(x))
df_deals_raw_rel['3'] = df_deals_raw_rel.target_name.apply(lambda x: dict_3.get(x))
df_deals_raw_rel['4'] = df_deals_raw_rel.target_name.apply(lambda x: dict_4.get(x))
df_deals_raw_rel['comb'] = df_deals_raw_rel.apply(lambda x: set.union(x['2'], x['3'], x['4']), axis = 1)
df_deals_raw_rel['comb_str'] = df_deals_raw_rel.comb.apply(lambda x: list(map(str, x)))
df_deals_raw_rel['strs'] = df_deals_raw_rel.comb_str.apply(lambda x:  ",".join(list(x)))
df_deals_raw_rel.drop_duplicates(subset='target_name', keep="first",inplace=True)

df_deals_raw_work = df_deals_raw_rel[['target_name','target_ebit','target_revenue','strs']]

df_deals_raw_work.dropna(subset='target_name',inplace=True)
df_deals_raw_work['strs'] = df_deals_raw_work['strs'].str.replace(',',' ')
df_deals_raw_work[['target_ebit','target_revenue']] = df_deals_raw_work[['target_ebit','target_revenue']].replace(np.nan, 0)
df_deals_raw_work.reset_index().drop('index',axis=1)

df_deals_raw_work['strs'] = df_deals_raw_work['strs'].apply(remove_punctuations) 
df_deals_raw_work['strs'] = df_deals_raw_work['strs'].apply(lambda x: x.lower())

stop_words = set(stopwords.words('german')) 

for name_de in df_deals_raw_work['strs']:    
    word_tokens = word_tokenize(name_de) 
    name_de = [w for w in word_tokens if not w in stop_words] 
    
scaler = MinMaxScaler()
df_deals_raw_work_fin = pd.DataFrame(scaler.fit_transform(
                        df_deals_raw_work.drop(columns=['target_name','strs'])),
                                    columns=['target_ebit', 'target_revenue'])

df_deals_raw_work_fin = df_deals_raw_work_fin.reset_index().drop('index',axis=1)

df_deals_raw_work = df_deals_raw_work.reset_index().drop('index',axis=1)
df_deals_raw_work1 = pd.concat([df_deals_raw_work.drop(columns=['target_ebit','target_revenue']), 
                                df_deals_raw_work_fin], axis=1)

vec_tf_target = TfidfVectorizer()
x_tf_target = vec_tf_target.fit_transform(df_deals_raw_work1['strs'])
df_wrds_tf_target = pd.DataFrame(x_tf_target.toarray(), columns=vec_tf_target.get_feature_names_out())
df_tf_target = pd.concat([df_deals_raw_work1, df_wrds_tf_target], axis=1)
df_tf_target.drop('strs', axis=1, inplace=True)
df_tf_target_4pca = df_tf_target.drop('target_name',axis=1)

# pca_tf_target = PCA()
# df_tf_target_pca = pca_tf_target.fit_transform(df_tf_target_4pca)
# total_explained_variance = pca_tf_target.explained_variance_ratio_.cumsum()
# n_over_95 = len(total_explained_variance[total_explained_variance>=.95])
# n_to_reach_95 = df_tf_target_pca.shape[1] - n_over_95
# pca_tf_target = PCA(n_components=n_to_reach_95)
# df_tf_target_pca = pca_tf_target.fit_transform(df_tf_target_4pca)

pca_tf_target = PCA(0.95)
df_tf_target_pca = pca_tf_target.fit_transform(df_tf_target_4pca)

nn_target_tf_pca = NearestNeighbors(n_neighbors=10).fit(df_tf_target_pca)

/tmp/ipykernel_13562/372696119.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_deals_raw_work.dropna(subset='target_name',inplace=True)
/tmp/ipykernel_13562/372696119.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_deals_raw_work['strs'] = df_deals_raw_work['strs'].str.replace(',',' ')
/tmp/ipykernel_13562/372696119.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/sta

In [23]:
df_tf_target_pca.shape

(1084, 844)

In [25]:
df_tf_target_4pca.shape

(1084, 4539)

In [24]:
np.savetxt('X_trans_test_jup.csv', df_tf_target_4pca, delimiter=",")

## Investors

In [18]:
invest_key['name_de'] = invest_key['name_de'].replace(dict(zip(key_match.name_de,key_match.new_keyword)))
invest_key_small = invest_key[['name','name_de']]
invest_key_small.drop_duplicates(inplace=True)
invest_key_concat = invest_key_small.astype(str).groupby('name').agg({'name_de':', '.join})
invest_key_concat1 = invest_key_concat['name_de'].str.replace('nan','').reset_index()
invest_key_concat1['name_de'] = invest_key_concat1['name_de'].apply(remove_punctuations)        
invest_key_concat1['name_de'].replace(r'^\s*$',np.nan,regex=True,inplace=True)
invest_key_concat1 = invest_key_concat1.dropna().reset_index().drop('index',axis=1)
invest_key_concat1['name_de'] = invest_key_concat1['name_de'].apply(lambda x: x.lower())

stop_words = set(stopwords.words('german')) 

for name_de in invest_key_concat1['name_de']:    
    word_tokens = word_tokenize(name_de) 
    name_de = [w for w in word_tokens if not w in stop_words] 
    
vec_tf_key = TfidfVectorizer()
x_tf_key = vec_tf_key.fit_transform(invest_key_concat1['name_de'])
df_wrds_tf_key = pd.DataFrame(x_tf_key.toarray(), columns=vec_tf_key.get_feature_names_out())
df_tf_key = pd.concat([invest_key_concat1, df_wrds_tf_key], axis=1)
df_tf_key.drop('name_de', axis=1, inplace=True)
df_tf_key_4pca = df_tf_key.drop('name',axis=1)

# pca_tf_key = PCA()
# df_tf_key_pca = pca_tf_key.fit_transform(df_tf_key_4pca)
# total_explained_variance = pca_tf_key.explained_variance_ratio_.cumsum()
# n_over_95 = len(total_explained_variance[total_explained_variance>=.95])
# n_to_reach_95 = df_tf_key_pca.shape[1] - n_over_95
# pca_tf_key = PCA(n_components=n_to_reach_95)
# df_tf_key_pca = pca_tf_key.fit_transform(df_tf_key_4pca)

pca_tf_key = PCA(0.95)
df_tf_key_pca = pca_tf_key.fit_transform(df_tf_key_4pca)

qty_of_nn = 5
nn_key_tf_pca = NearestNeighbors(n_neighbors=qty_of_nn).fit(df_tf_key_pca)

/tmp/ipykernel_13562/528286751.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  invest_key_small.drop_duplicates(inplace=True)


In [19]:
df_tf_key_4pca.head()

,3d,3ddruck,abfall,abfüllen,abgas,absorption,abwasser,abwasserbereich,acker,advisory,agentur,agnostisch,agrar,akadamie,aktien,aktiv,akustik,alarm,algen,alkohol,altenheim,altenpflege,aluproduktion,amazon,analyse,analysegerät,andere,angnostisch,anlage,anlagebau,anlagekomponenten,anlagen,anlagenplanung,antrieb,antriebssystem,anwendungen,anästhesie,apotheke,app,apparat,apparel,arbeitnehmer,arbeitskleidung,arbeitsschutz,architektur,archivierung,armaturentechnik,armee,artificial,arzneimittel,arzt,asyl,auch,audio,aufbereitung,auktionshaus,ausbau,ausbaudienst,ausbildung,auslagerung,auspuffsystem,ausrüstung,authentifizierung,automatenunternehmen,automationstechnik,automatisierung,automatisierungstechnik,automobil,automobilelektronik,automobilindustrie,automobilkomponenten,automobilversicherung,automobilwerkstatt,automobilzulieferer,autonom,außenbereich,außenteile,avionics,b2b,b2c,baby,babynahrung,backen,backup,bad,bahn,bahntechnik,bank,batterie,batteriespeichersysteme,bau,bauchemie,bauelemente,baugruppe,bauhandel,bauindustrie,bauleitung,baumaschine,baumaschinen,baumaterial,baustelle,baustoffe,bauteil,bautenschutz,bauträger,bauunternehmen,bauwerk,bauwirtschaft,bauzulieferer,bearbeitung,beatmung,bedachung,befestigungstechnik,bekleidung,beleuchtung,benutzerfreundlichkeit,benutzeroberfläche,bequem,beratung,bergbau,berstlining,berufsausbildung,beschichtung,beschichtungstechnik,beschickung,beschlag,beschleuniger,bestandshalter,bestandsimmobilien,bestandsmanagement,beton,betonelemente,betreuung,betreuungsangebot,betriebsausstattung,betriebsmedizin,betriebsmittel,betriebssystem,bett,bewertung,biegetechnik,bildung,bildungsakademie,bio,bioartikel,biokraftstoff,biologie,biomasse,biomaterial,bioprodukte,biotechnik,biotechnologie,biowissenschaften,blech,blechbearbeitung,blenden,blitzschutz,blockchain,boden,bodenbelag,bodenheizung,bodenverlegung,bodenzug,bohren,bohrtechnik,boot,boutique,brandschutz,brauerei,breitband,broker,brückenbau,buch,buchbinder,buchhaltung,business,businessinteligence,bäcker,büro,büroausstattung,cad,callcenter,cannabis,carport,cbd,cell,chain,chat,chemie,chirugie,chirurgie,cloud,cnc,co2,computer,container,cybersicherheit,d2c,daas,dach,dachdecker,data,datealyse,daten,datenerfassung,datensicherheit,datenverarbeitung,dcommerce,decken,dekoration,denkmalschutz,design,desktop,detektor,devices,diagnose,dialyse,dichtungen,diebstahlschutz,diensteistungen,dienstleistung,dienstleistungen,digitalisierung,direktbeteiligung,discounter,display,diy,dna,dokument,dosiereinrichtungen,draht,drahtlos,drehmaschine,drogerien,drohne,druck,druckbehälter,drucker,dünger,ecommerce,economy,einbau,einkaufszentren,einnahmen,einrichtung,einweisung,einzelhandel,einzelpflege,eisenbahn,eismaschine,elastomere,elektroartikel,elektroinstallation,elektromobilität,elektronik,emobilität,endoskop,energie,engineering,entkarbonisierung,entkeimung,entlackung,entsorgung,entwässerung,equipment,ergonomik,erholung,erlebnis,ernährung,erp,erwachsenenbildung,esg,etabliert,etc,etikett,event,experte,factory,fahren,fahrrad,fahrradverleih,fahrräder,fahrzeug,farbe,fassade,fba,feinmechanik,fenster,fertigbau,fertighaus,fertigung,fertigungsspezialist,fertigungstechnologie,feuerwehr,film,filmproduktion,filter,filtrierung,fintech,firmenkurse,fisch,fischzucht,fitness,fizdienstleistung,fizen,fizierung,fizinstitut,fleisch,fliegen,flotte,flug,fluggesellschaften,flughafen,flugzeug,flugzeughersteller,fluid,flüchtlinge,flüssigkeit,folien,fond,fondsverwaltung,forderungsmanagement,formenbau,formtechnik,forschung,fortbildung,foto,franchise,freiberufler,freileitung,freizeit,frequenumrichter,frequenzartikel,fugenabdichtung,fuhrpark,fulfillment,funk,futter,fördertechnik,galvanik,garagen,garten,gas,gastronomie,gebraucht,gebäude,gebäudeautomation,gebäudetechnik,gefahrenstoffe,geflügel,gefragt,geld,generalist,generalunternehmer,generator,genetik,genossenschaft,geotechnik,gerüstbau,geschenk,geschirr,geschäftsdienste,geschäftsführung,geschäftsprozess,gestaltung,gesund,getriebe,getränk,gewerbebau,gewindetei

In [20]:
df_tf_key_pca.shape

(3184, 391)

# Prediction

In [22]:
target_pred = df_deals_raw_work1[df_deals_raw_work1['target_name']=='Maybach-Luxury']

target_pred_scapled = pd.DataFrame(scaler.transform(target_pred.drop(columns=['target_name','strs'])),
                                  columns=['target_ebit', 'target_revenue'])

tx_to_pred = vec_tf_target.transform(target_pred['strs'])
df_target_pred_wrds = pd.DataFrame(tx_to_pred.toarray(),columns=vec_tf_target.get_feature_names_out())
df_target_pred = df_target_pred_wrds.copy()
df_target_pred.insert(0,'target_ebit',target_pred_scapled['target_ebit'])
df_target_pred.insert(1,'target_revenue',target_pred_scapled['target_revenue'])
df_target_pred_pca = pca_tf_target.transform(df_target_pred)

nn_targets = nn_target_tf_pca.kneighbors(df_target_pred_pca)

name = []
description = []
distance = []



for x,y in zip(nn_targets[1][0],nn_targets[0][0]):
    name.append(df_deals_raw_work1['target_name'].iloc[x])
    description.append(df_deals_raw_work1['strs'].iloc[x])
    distance.append(y)
    
    
df_companies = pd.DataFrame({'name':name,
               'description':description,
               'distance':distance})


matching_investors = []
matching_target = []
matching_distance = []
for company in df_companies['name']:
    next_investor = matching_table[(matching_table['target_name']==company) & (matching_table['deal_stage_id']==4)]['comp_name'].tolist()
    matching_investors+=next_investor
    matching_target+=len(next_investor)*[company]
    next_distance = df_companies[df_companies['name']==company]['distance'].tolist()
    matching_distance+=len(next_investor)*next_distance
df_match_investors = pd.DataFrame({'investors':matching_investors,'targets':matching_target,'distance':matching_distance})


if len(df_match_investors['investors'].unique())>=10:
    best_investors = df_match_investors['investors'].unique()[:10].tolist()
else:
    best_investors = df_match_investors['investors'].unique().tolist()
    
name_investor = []
description_investor = []
distance_investor_investor = []
distance_target_target = []


for investor in best_investors:
    name_investor.append(investor)
    if invest_key_concat1['name'].str.contains(investor).any():
        description_investor.append(invest_key_concat1[invest_key_concat1['name']==investor]['name_de'].to_list()[0])
    else:
        description_investor.append('Investor not in the list')
    distance_investor_investor.append(0)
    distance_target_target.append(df_match_investors[df_match_investors['investors']==investor]['distance'].min())


for investor in best_investors:
    if invest_key_concat1['name'].str.contains(investor).any():
        first_distance = df_match_investors[df_match_investors['investors']==investor]['distance'].min()
        to_pred = invest_key_concat1[invest_key_concat1['name']==investor]
        x_to_pred = vec_tf_key.transform(to_pred['name_de'])
        df_pred_wrds = pd.DataFrame(x_to_pred.toarray(),columns=vec_tf_key.get_feature_names_out())
        df_pred = pd.concat([to_pred,df_pred_wrds], axis=1)
        df_pred.drop('name_de', axis=1, inplace=True)
        df_pred_4pca = df_pred.drop('name',axis=1).dropna()
        df_pred_pca = pca_tf_key.transform(df_pred_4pca)

        nn_investors = nn_key_tf_pca.kneighbors(df_pred_pca)


        for x,y in zip(nn_investors[1][0],nn_investors[0][0]):
            name_investor.append(invest_key_concat1['name'].iloc[x])
            description_investor.append(invest_key_concat1['name_de'].iloc[x])
            distance_investor_investor.append(y)
            distance_target_target.append(first_distance)
        
df_investors = pd.DataFrame({'name':name_investor,
               'description':description_investor,
               'distance_investor<=>investor':distance_investor_investor,
                'distance_target<=>target':distance_target_target,
                'distance_target<=>investor': [a+b for a,b in zip(distance_investor_investor,distance_target_target)]})

df_investors_sorted = df_investors.sort_values('distance_target<=>investor')
df_investors_sorted.reset_index(inplace=True)
df_investors_sorted.drop('index',axis=1,inplace=True)

/tmp/ipykernel_3278/3289453189.py:57: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  if invest_key_concat1['name'].str.contains(investor).any():
/tmp/ipykernel_3278/3289453189.py:66: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  if invest_key_concat1['name'].str.contains(investor).any():


In [8]:
df_investors_sorted.head()

,name,description,distance_investor<=>investor,distance_target<=>target,distance_target<=>investor
0,Eso Capital UK Ltd,zubehör glas handel luxusgüter lizenz sme fizierung,0.000000,0.002793,0.002793
1,Eso Capital UK Ltd,zubehör glas handel luxusgüter lizenz sme fizierung,0.020072,0.002793,0.022865
2,Aprima,personaldienstleister sme,0.971242,0.002793,0.974035
3,CIT Leveraged Finance,fizierung fizierung fizierung übernahme,1.017259,0.002793,1.020052
4,Essence Capital,bewertung fizierung strategic advisory fond,1.017392,0.002793,1.020185


In [9]:
import joblib

In [17]:
joblib.dump(nn_target_tf_pca,'model_investors.joblib')  

['model_investors.joblib']

In [18]:
pipe_targets = joblib.load('model_investors.joblib')

In [19]:
df = pd.read_csv('../DealMatch/target_data.csv')

In [20]:
nearest_targets = pipe_targets.kneighbors(df,10)

/home/pawel/.pyenv/versions/3.8.12/envs/dealmatch/lib/python3.8/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but NearestNeighbors was fitted without feature names
  warnings.warn(


ValueError: could not convert string to float: '876 Berlin'

In [15]:
pipe_targets

(array([[0.00279297, 1.1119553 , 1.11658997, 1.13265075, 1.13274887,
         1.15757688, 1.16336   , 1.17447485, 1.179343  , 1.18643689]]),
 array([[  75,   74,  500, 1035, 1058,  843,  858,  588, 1008,  366]]))

In [21]:
df_deals_raw_work1.head()

,target_name,strs,target_ebit,target_revenue
0,Sun [Target],energie solar solarenergie pv energie photovoltaik,0.617200,0.000140
1,Supple [Target],solarenergie pv sonne solar energie erneuerbare energie sonstige energie fresnel photovoltaik,0.617200,0.028986
2,SPH Sustainable Process Heat GmbH,abwärme industrielle produkte dienstleistungen pumpen und armaturen wärmepumpentechnologie prozesswärme energieeffizient pumpen niedertemperatur,0.607248,0.000355
3,SUMMIQ AG,infrastrukturfonds finanzdienstleistungen stadtwerkegruppen energieversorger finanzdienstleistungen sonstige stadtwerke,0.617200,0.000140
4,KSW Bioenergie GmbH,erneuerbare energie bioenergie bioenergie biomasse co2 kraftwerk energie nachhaltigkeit energie biotreibstoff,0.617200,0.000140
